# Scikit-Learn Linear Regression

### This implementation uses Scikit-Learn to build a linear regression model

## Goal: Predict used car prices and compare with No-Framework implementation

What Scikit-Learn provides (what we built manually in No-Framework):
- `train_test_split()`: Replaces our manual `train_test_split()` function
- `StandardScaler`: Replaces our manual `compute_scaling_params()` and `scale_features()` functions
- `LinearRegression`: Replaces our manual `foward()`, `compute_cost()`, `compute_gradients()`, and `train()` functions
- `mean_squared_error`, `r2_score`: Replaces our manual metric functions

Key Differerence: Scikit-Learn uses the **Normal Equation** (closed-form solution) instead of gradient descent. This means:
- No learning rate to tune
- No iterations to set
- Instant optimal solution (for datasets that fit in memory)



# Imports

In [2]:
# pandas: Data manipulation library for loading CSV files into dataframes
# We use pandas instead of numpy.genfromtxt() for easier column access
import pandas as pd

# numpy: Numerical operations library
# Still needed for array operations and setting random seed
import numpy as np

# matplotlib.pyplot: Plotting library for creating visualizations
import matplotlib.pyplot as plt

# os: Operating system interface for file path handling
import os

# Scikit-Learn Imports

# train_test_split: Splits array into random train and test subsets
from sklearn.model_selection import train_test_split

# StandardScaler: Standardizes features using z-score normalization
from sklearn.preprocessing import StandardScaler

# LinearRegression: Ordinary leas squares Linear Regression
# KEY DIFFERENCES: Uses normal Equation instead of gradient descent
from sklearn.linear_model import LinearRegression

# mean_squared_error: Calculates MSE between predictions and actual values
from sklearn.metrics import mean_squared_error, r2_score

# Set random seed for reproducabiility (matches our project-wide seed)
RANDOM_SEED = 113
np.random.seed(RANDOM_SEED)

print("All imports successful!")
print(f"Random seed set to: {RANDOM_SEED}")

All imports successful!
Random seed set to: 113
